In [7]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Lasso,LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm 
from sklearn.metrics import r2_score
import pickle

In [8]:
df =pd.read_csv('Admission_Predict.csv')
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [9]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [10]:
df.drop(columns=['Serial No.'],inplace=True)

In [11]:
df.describe()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


## Splitting Features into Dependent & Independent Features, Data into Train, Test

In [12]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [13]:
x.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [14]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=45)

## Standardizing the Dependent Features data using Z-Transform

In [16]:
scalar=StandardScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.transform(x_test)

In [17]:
pickle.dump(scalar,open("scaling_model.pkl","wb"))

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df=pd.DataFrame()
vif_df['vif']=[variance_inflation_factor(x_train,i) for i in range(x_train.shape[1])]
vif_df['feature']=x.columns
vif_df

,vif,feature
0,4.609846,GRE Score
1,4.457171,TOEFL Score
2,3.071257,University Rating
3,3.093621,SOP
4,2.576956,LOR
5,5.160770,CGPA
6,1.505983,Research


In [19]:
linear=LinearRegression()
linear.fit(x_train,y_train)

LinearRegression()

In [20]:
predict_1=linear.predict(x_test)
predict_1=predict_1.reshape(-1,1)
predict_1

array([[0.69264093],
       [0.51204317],
       [0.73193591],
       [0.96833792],
       [0.6961389 ],
       [0.80896423],
       [0.65401816],
       [0.74222942],
       [0.61723538],
       [0.84045399],
       [0.62270716],
       [0.87367668],
       [0.69061046],
       [0.98122216],
       [0.68338436],
       [0.6352006 ],
       [0.49532814],
       [0.96244479],
       [0.77166362],
       [0.89307479],
       [0.74086036],
       [0.68281416],
       [0.55813106],
       [0.65442459],
       [0.95133775],
       [0.87608186],
       [0.57006913],
       [0.49634946],
       [0.70901898],
       [0.66798761],
       [0.82706863],
       [0.85591726],
       [0.96954803],
       [0.85022733],
       [0.77936033],
       [0.60890881],
       [0.73474673],
       [0.78925785],
       [0.63580307],
       [0.47095921],
       [0.6289702 ],
       [0.70279647],
       [0.78547459],
       [0.58334233],
       [0.71191383],
       [0.77617327],
       [0.72232157],
       [0.723

In [21]:
linear.score(x_test,y_test)

0.8049825387312869

In [22]:
## Adjusted R2_Score

def adjr2_score(x,y):
    r2=linear.score(x,y)
    n=x.shape[0]
    p=x.shape[1]
    adj = 1-(1-r2)*(n-1)/(n-p-1)
    return adj

In [23]:
adjr2_score(x_test,y_test)

0.7860225077746065

In [24]:
linear.intercept_

0.7250624999999998

In [25]:
linear.coef_

array([ 0.02451747,  0.01322677,  0.00614046, -0.00127564,  0.01778   ,
        0.07352879,  0.01144109])

## Regularization

In [26]:
lassocv=LassoCV(alphas=None,cv=100,max_iter=200000)
lassocv.fit(x_train,y_train)

LassoCV(cv=100, max_iter=200000)

In [27]:
lassocv.alpha_

0.0005494605103436922

## We are able to calculate the hyperparameter alpha by performing cross validation 

In [28]:
lasso_l1=Lasso(alpha=lassocv.alpha_)
lasso_l1.fit(x_train,y_train)


Lasso(alpha=0.0005494605103436922)

In [29]:
lasso_l1.score(x_test,y_test)

0.8045032883955116

In [30]:
ridgecv=RidgeCV(alphas=np.random.uniform(0,10,50),cv=50)
ridgecv.fit(x_train,y_train)

RidgeCV(alphas=array([4.68255464, 5.35168789, 2.30240988, 1.57456526, 8.70642438,
       6.18553581, 4.58497915, 3.79356657, 8.8511423 , 8.27966132,
       4.72659274, 5.08174892, 7.98698305, 8.54439191, 8.23673547,
       2.78318151, 6.55855235, 3.76559049, 6.48783631, 8.24515787,
       0.2772481 , 2.77533212, 0.78396892, 2.81300161, 7.76715178,
       9.49400489, 8.45325982, 3.29011446, 8.13307115, 7.71276643,
       4.27495365, 5.3303028 , 0.8842858 , 8.21551517, 4.84895905,
       0.13712534, 5.15206201, 8.76108056, 8.47583053, 5.2337354 ,
       8.55462747, 6.32108501, 1.96256755, 7.46400771, 5.44390477,
       7.05253282, 3.97819146, 3.34341295, 9.20437282, 9.58416302]),
        cv=50)

In [31]:
ridgecv.alpha_

4.2749536502437895

In [32]:
ridge_l2=Ridge(alpha=ridgecv.alpha_)
ridge_l2.fit(x_train,y_train)

Ridge(alpha=4.2749536502437895)

In [33]:
ridge_l2.score(x_test,y_test)

0.805628249780225

In [34]:
elasticcv=ElasticNetCV(alphas=None,cv=20)
elasticcv.fit(x_train,y_train)

ElasticNetCV(cv=20)

In [35]:
elasticcv.alpha_

0.0013548011972587893

In [36]:
elasticcv.l1_ratio_

0.5

In [37]:
elastic_lr=ElasticNet(alpha=elasticcv.alpha_,l1_ratio=elasticcv.l1_ratio_)
elastic_lr.fit(x_train,y_train)


ElasticNet(alpha=0.0013548011972587893)

In [38]:
elastic_lr.score(x_test,y_test)

0.8045808179493981

In [39]:
pickle.dump(ridge_l2, open("ridge_regression_model.pkl","wb"))